In [8]:
import requests

BASE_URL = "http://localhost:8000"

# Start a session so cookies persist between requests
session = requests.Session()

# 1. Fetch CSRF token (and laravel_session cookie)
resp = session.get(f"{BASE_URL}/csrf-token")
print("CSRF token response:", resp.json())
csrf_token = resp.json()["token"]

# 2. Perform login using that token
login_payload = {
    "email": "test@example.com",
    "password": "password123"
}

headers = {
    "X-CSRF-TOKEN": csrf_token,
    "Accept": "application/json"
}

resp = session.post(f"{BASE_URL}/login", json=login_payload, headers=headers)
print("Login status code:", resp.status_code)
print("Login response JSON:", resp.json())


CSRF token response: {'token': 'osjTGSUCvoDY216GUvfudiOsJKTH7Kr01hECMfe3'}
Login status code: 422
Login response JSON: {'message': 'The provided credentials are incorrect.', 'errors': {'email': ['The provided credentials are incorrect.']}}


In [9]:
import requests
import time

# Base URL of your Laravel app
base_url = "http://localhost:8000"

# Get CSRF token first (since you're using web middleware)
session = requests.Session()

# Get CSRF token
csrf_response = session.get(f"{base_url}/csrf-token")
csrf_token = csrf_response.json()['token']

print(f"CSRF Token: {csrf_token}")

# Test data
login_data = {
    'email': 'test@example.com',
    'password': 'wrongpassword'
}

# Headers
headers = {
    'X-CSRF-TOKEN': csrf_token,
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Type': 'application/json'
}

print("\nTesting rate limiting...")
print("=" * 40)

# Try 7 login attempts
for i in range(1, 8):
    print(f"Attempt {i}: ", end="")
    
    try:
        response = session.post(
            f"{base_url}/login",
            json=login_data,
            headers=headers
        )
        
        print(f"Status: {response.status_code}", end="")
        
        if response.status_code == 429:
            print(" - RATE LIMITED! ✅")
            print(f"Response: {response.text}")
            break
        elif response.status_code == 422:
            print(" - Validation error (expected)")
        elif response.status_code == 419:
            print(" - CSRF token expired")
            break
        else:
            print(f" - {response.reason}")
            
    except Exception as e:
        print(f"Error: {e}")
    
    time.sleep(0.5)  # Small delay between requests

print("\nTest completed!")

CSRF Token: tf2on4BZJZ3Wyy8XCz3LJvC2fuFAkAav74zIGki7

Testing rate limiting...
Attempt 1: Status: 422 - Validation error (expected)
Attempt 2: Status: 422 - Validation error (expected)
Attempt 3: Status: 422 - Validation error (expected)
Attempt 4: Status: 422 - Validation error (expected)
Attempt 5: Status: 422 - Validation error (expected)
Attempt 6: Status: 429 - RATE LIMITED! ✅
Response: {
    "message": "Too Many Attempts.",
    "exception": "Illuminate\\Http\\Exceptions\\ThrottleRequestsException",
    "file": "C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Routing\\Middleware\\ThrottleRequests.php",
    "line": 244,
    "trace": [
        {
            "file": "C:\\Users\\Jandrik\\Documents\\MyFILES\\TUA ITSE 4th Year\\IAS 2\\sample-app-ITPC174\\backend\\vendor\\laravel\\framework\\src\\Illuminate\\Routing\\Middleware\\ThrottleRequests.php",
            "line": 154,
            "function